In [1]:
from sdf import *
import pyvista as pv
from scipy.interpolate import splev, splprep
from scipy.optimize import minimize_scalar
import mcubes
from tree_viewer import *
import os
import random
from tqdm import tqdm
import trimesh


In [2]:
folder=""
file = ".npy"
print(file)

data = np.load(folder + "/" + file)
serial = list(data.flatten())

tree, _ = deserialize_pre_order_k(serial, k = 39)


.npy


FileNotFoundError: [Errno 2] No such file or directory: '/.npy'

In [ ]:
def create_3d_spline(points, s=0):

    """
    Creates a 3D spline from control points.
    control_points: array of shape (N, 3)
    s: smoothing factor
    """
    k = 3
    if len(points) <= 3: k = len(points) - 1

    tck, _ = splprep(points.T, s=s, k=k)
    return tck


def sample_spline(tck, num_samples):

    t = np.linspace(0, 1, num_samples)
    x, y, z = splev(t, tck)
    
    return np.vstack((x, y, z)).T

In [ ]:
def get_branches(tree):

    if tree is None:
        return []
    
    branches = []
    
    def dfs(node, path):
        if node is None:
            return
        path.append(node.data)
        
        if node.left is None and node.right is None:
            branches.append(path[:])  # Se guarda una copia del camino actual
        
        if node.left:
            dfs(node.left, path)
        if node.right:
            dfs(node.right, path)
        
        path.pop()  # Retroceder para explorar otros caminos
    
    dfs(tree, [])

    new_branches = []
    for branch in branches:
        
        l = []
        for node in branch:
            l.append([node["x"], node["y"], node["z"]] + node["r"])

        l = np.array(l)
        new_branches.append(l)

    return new_branches

def draw_branches(branches):

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(tree.data["x"], tree.data["y"], tree.data["z"], marker = "*", s = 50, c = "g")

    for branch in branches:

        color = np.random.rand(3)

        for i in range(len(branch) - 1):
            ax.plot([branch[i][0], branch[i+1][0]],[branch[i][1], branch[i+1][1]],[branch[i][2], branch[i+1][2]], c = color)

In [ ]:
def sdf_tree(tree, vesselfunc):


    # Step 1: Get branches from the tree
    branches = get_branches(tree)

    # Step 2: Draw the branches (possibly for visualization)
    draw_branches(branches)

    vessels = []

    # Step 3: Process each branch
    for i, branch in enumerate(branches):
        nodes = branch[:, :3]
        splines = branch[:, 3:]

        # Step 4: Create a 3D spline from the node coordinates
        tck = create_3d_spline(nodes)

        # Step 5: Sample 100 points along the spline
        sampled_points = sample_spline(tck, num_samples=100)

        # Step 6: Create a vessel3 object and add it to the list
        vessels.append(vesselfunc(sampled_points, nodes, splines))

    # Step 7: Combine all vessels into a single shape (union operation)
    result = union(*vessels)


    return result, branches


def sdf_tree_martin(tree):
    return sdf_tree(tree, vessel3)


In [ ]:
def distance_to_spline(t, p):
    """
    Computes Euclidean distance between point p and spline at parameter t.
    """
    spline_point = np.array(splev(t, tck))
    return np.linalg.norm(p - spline_point)

# -------------------------------------------------------------------------------


In [3]:
f,_ = sdf_tree_martin(tree)
bounds = f.save("model.stl", samples = 2 ** 5, sparse = True)# 2 **18
bounds = f.save("model.stl", samples = 2 ** 18, bounds = bounds, sparse = True)# 2 **18

mesh = pv.read("model.stl")
plotter = pv.Plotter()
plotter.add_mesh(mesh, color="red", show_edges=False)
plotter.add_axes()
plotter.show()

NameError: name 'sdf_tree_martin' is not defined